In [87]:
# Install, import libraries
import sys
!{sys.executable} -m pip install google-api-python-client earthengine-api

import xarray as xr
import rasterio

In [84]:
# Import, set up Earth Engine
import ee
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/wwHC1PsOO0MensyqfJ0hqpBlDb92EhFNmC1qP4ePsNsF0CG_wh-nf2U



Successfully saved authorization token.


In [52]:
# Create a polygon over Chapel Hill
chapel_hill = ee.Geometry.Polygon([
    [-79.0899192833252, 35.928421884080535],
    [-79.02846451037598, 35.92897789378388],
    [-79.02777786486817, 35.891716600077054],
    [-79.08923263781739, 35.89227286777993],
])

polygon = ee.Geometry.Polygon([
  [[-5, 40], [65, 40], [65, 60], [-5, 60], [-5, 60]]
]);

## Set up landsat
landsat = (ee.ImageCollection('LANDSAT/LE07/C01/T1')
           .filterDate('2019-01-01', '2019-12-01')
           .filterBounds(chapel_hill)
           .filter(ee.Filter.gte('SUN_ELEVATION',50))
          )


# Composite a collection
landsat_composite = ee.Algorithms.Landsat.simpleComposite(landsat)

## Display in notebook
from IPython.display import Image

Image(url = landsat_composite
      .getThumbURL({
        'bands': ['B3', 'B2', 'B1'],
        'region': chapel_hill,
        'min': 0,
        'max': 75,
        'dimensions': 1080
    })
)

In [100]:
def search_landsat(collection, year, lat, long, width):

    polygon = ee.Geometry.Polygon([
            [lat, long],
            [lat+width, long],
            [lat+width, long+width],
            [lat, long+width],

        ])
    
## Set up landsat
    landsat = (ee.ImageCollection(collection)
               .filterDate(year+'-01-01', year+'-12-01')
               .filterBounds(polygon)
               .filter(ee.Filter.gte('SUN_ELEVATION',50))
              )
    
    landsat_composite = ee.Algorithms.Landsat.simpleComposite(landsat)

## Display in notebook
    from IPython.display import Image

    img = Image(url = landsat_composite
        .getThumbURL({
            'bands': ['B3', 'B2', 'B1'],
            'region': polygon,
            'min': 0,
            'max': 75,
            'dimensions': 1080
        })
   )
    return img

chapel_hill = search_landsat('LANDSAT/LE07/C01/T1', '2019', -79.0899192833252, 35.928421884080535, 0.15)

chapel_hill

In [74]:
chapel_hill = search_landsat('LANDSAT/LE07/C01/T1', '2019', -79.0899192833252, 35.928421884080535, 0.3)

chapel_hill

In [98]:
def search_landsat_data(collection, year, lat, long, width):

    polygon = ee.Geometry.Polygon([
            [lat, long],
            [lat+width, long],
            [lat+width, long+width],
            [lat, long+width],

        ])
    
## Set up landsat
    landsat = (ee.ImageCollection(collection)
               .filterDate(year+'-01-01', year+'-12-01')
               .filterBounds(polygon)
               .filter(ee.Filter.gte('SUN_ELEVATION',50))
              )
    
    landsat_composite = ee.Algorithms.Landsat.simpleComposite(landsat)


    env = rasterio.Env(GDAL_DISABLE_READDIR_ON_OPEN='EMPTY_DIR',
                       CPL_VSIL_CURL_USE_HEAD=False,
                       CPL_VSIL_CURL_ALLOWED_EXTENSIONS='TIF')


    url = landsat.first().getDownloadURL({
        'scale': 30,
        'crs': 'EPSG:3358',
        'region': polygon,
       
    })

    return xr.open_rasterio(url)
    
search_landsat_data('LANDSAT/LE07/C01/T1', '2019', -79.0899192833252, 35.928421884080535, 0.3)

RasterioIOError: HTTP response code: 404

In [ ]:
def create_multiband_dataset(bands=['B3','B2','B1'], chunks={'band': 1, 'x': xchunk, 'y': ychunk}):
    '''A function to load multiple landsat bands into an xarray dataset 
    
    '''
    # Each image is a dataset containing both band4 and band5
    datasets = []
    for band in bands:
        url = row[band]
        with env:
            da = xr.open_rasterio(url, chunks=chunks)
        da = da.squeeze().drop(labels='band')
        ds = da.to_dataset(name=band)
        datasets.append(ds)

    ds = xr.merge(datasets)
    
    return ds